## Imports

In [23]:
import pandas as pd
import re
import numpy as np
import json

def merge_utts(utts):
  merged = []

  last_speaker = int(utts[0][0])
  cur_utt = utts[0]

  for utt in utts[1:]:
    speaker = int(utt[0])
    if speaker != last_speaker:
      merged.append(cur_utt)
      cur_utt = utt
    else:
      cur_utt += f"\n{utt[2:].strip().capitalize()}"

    last_speaker = speaker

  if merged[-1] != cur_utt:
    merged.append(cur_utt)

  return merged

def clean_dialogue(dialogue):
  cleaned = re.sub(r"<[^>]*>", ' ', dialogue)
  cleaned = [i.strip() for i in cleaned.split("Пользователь") if len(i.strip())>0]

  if not cleaned[0][0].isalnum():
    cleaned.pop(0)
    if not cleaned[-1][-1].isalnum():
      cleaned[-1] = cleaned[-1][:-2]

  return cleaned

def clean_persona(persona):
  persona = persona.split("<br />")
  persona = [re.sub(r"<[^>]*>", ' ', trait).strip() for trait in persona]
  persona = [i.strip() for i in persona if len(i.strip())>0]
  return persona

def preprocess(ds, bot_prefix="Пользователь 2",user_prefix="Пользователь 1"):
  fid_dialogue = []

  for i in range(ds.shape[0]):
    entry = ds.iloc[i]
    persona1 = entry['persona_1_profile']
    persona2 = entry['persona_2_profile']
    personas = [persona1,persona2]

    dialogue = entry['dialogue']

    cleaned = clean_dialogue(dialogue)

    merged = merge_utts(cleaned)
    merged.insert(0,"")

    personas_order = [int(merged[1][0]),int(merged[2][0])]

    for i_per in range(len(personas)):
      persona_num = personas_order[i_per]
      persona = personas[persona_num-1]

      bot_personality = clean_persona(persona)

      dialogue_history = []

      for j in range(i_per,len(merged)-1,2):
        dialogue_entry = {"id":i}

        question = f"{user_prefix}:{merged[j][2:]}"
        answer = f"{bot_prefix}:{merged[j+1][2:]}"

        dialogue_entry["question"] = question
        dialogue_entry["target"] = answer
        dialogue_entry["answers"] = [answer]

        history_string = "\n".join(dialogue_history)

        dialogue_entry["ctxs"] = [{"title":trait,"text":history_string} for trait in bot_personality]

        fid_dialogue.append(dialogue_entry)

        dialogue_history.extend([question,answer])

  return fid_dialogue

## Load data

In [ ]:
!unzip TlkPersonaChatRus.zip -d ./

In [ ]:
ru_dialogues = pd.read_csv('./TlkPersonaChatRus/dialogues.tsv', sep='\t')

In [ ]:
ru_dialogues.head()

,persona_1_profile,persona_2_profile,dialogue
0,<span class=participant_1>У меня любимая работ...,<span class=participant_2>Ищу принца.<br />Вед...,<span class=participant_2>Пользователь 2: Прив...
1,<span class=participant_1>Я работаю учителем<b...,<span class=participant_2>Я бизнесмен<br />У м...,<span class=participant_1>Пользователь 1: Прив...
2,<span class=participant_1>Я купила дом<br />Я ...,<span class=participant_2>Я пою в караоке<br /...,<span class=participant_1>Пользователь 1: Прив...
3,<span class=participant_1>я врач и женат<br />...,<span class=participant_2>Я мальчик<br />Я учу...,<span class=participant_2>Пользователь 2: Здра...
4,<span class=participant_1>Я школьница.<br />Я ...,<span class=participant_2>Я простоват.<br />Лю...,<span class=participant_1>Пользователь 1: Прив...


In [ ]:
ru_dialogues.shape

(10013, 3)

In [ ]:
# Devide data frame into the train, dev, test. Dev and test are of equal size

train_size = 0.8

df = ru_dialogues.copy()

ru_dialogues_train = df.sample(frac = train_size,random_state=42)
df = df.drop(ru_dialogues_train.index)
ru_dialogues_dev = df.sample(frac = 0.5,random_state=42)
ru_dialogues_test = df.drop(ru_dialogues_dev.index)

In [ ]:
print("train",ru_dialogues_train.shape)
print("dev",ru_dialogues_dev.shape)
print("test",ru_dialogues_test.shape)

train (8010, 3)
dev (1002, 3)
test (1001, 3)


In [14]:
for i in range(10):
  entry = ru_dialogues.iloc[i]
  persona1 = clean_persona(entry['persona_1_profile'])
  persona2 = clean_persona(entry['persona_2_profile'])

  dialogue = entry['dialogue']
  cleaned = clean_dialogue(dialogue)

  merged = merge_utts(cleaned)

  print("Persona 1:",persona1)
  print("Persona 2:",persona2)

  print("\nDialogue:")
  print(*merged,sep='\n')
  print("\n\n")

Persona 1: ['У меня любимая работа.', 'Я уважаю людей.', 'У меня есть животное.', 'У меня хороший друг.', 'Я люблю кофе.']
Persona 2: ['Ищу принца.', 'Веду активный образ жизни.', 'Люблю читать классику.', 'Выращиваю фиалки.', 'Люблю общение.']

Dialogue:
2: Привет) расскажи о себе
1: Привет) под вкусный кофеек настроение поболтать появилось )
2: Что читаешь? Мне нравится классика
Я тоже люблю пообщаться
1: Люблю животных, просто обожаю, как и свою работу)
Я фантастику люблю
2: А я выращиваю фиалки
И веду здоровый и активный образ жизни!
1: Ух ты, интересно.
2: Ты случайно не принц на белом коне? Я его очень жду ..
1: А у меня из хобби каждую неделю тусить с моим лучшим другом)



Persona 1: ['Я работаю учителем', 'У меня есть собака', 'Я люблю петь', 'Я живу сама', 'Я люблю цветы']
Persona 2: ['Я бизнесмен', 'У меня скоро свадьба', 'Меня любят только за деньги', 'Я не люблю людей', 'не люблю тупые опросы']

Dialogue:
1: Привет!
2: Привет,Как жизнь?
1: Отлично) Солнышко светит, птички 

## Process data

In [24]:
train = preprocess(ru_dialogues_train)

In [25]:
dev = preprocess(ru_dialogues_dev)

In [26]:
test = preprocess(ru_dialogues_test)

In [27]:
len(train)

134917

In [28]:
len(dev)

16970

In [29]:
len(test)

16896

## Save dataset

In [30]:
with open('train.json', 'w') as outfile:
    json.dump(train, outfile)

In [31]:
with open('dev.json', 'w') as outfile:
    json.dump(dev, outfile)

In [32]:
with open('test.json', 'w') as outfile:
    json.dump(test, outfile)